<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim


### Objetivo

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec

### Datos

In [3]:
df = pd.read_csv('debate_presidencial.txt', sep='/n', header=None)
df.head()

C:\Users\cbureu\AppData\Local\Temp\ipykernel_8672\3786229320.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('debate_presidencial.txt', sep='/n', header=None)


,0
0,"TEXTO DEL PRIMER DEBATE PRESIDENCIAL 2023, REA..."
1,"CONVENCIONES PROVINCIAL DE FORUM, CON SEDE EN ..."
2,"ESTERO, A CARGO DE LA UNIVERSIDAD NACIONAL DE ..."
3,DE 2023.
4,MODERADOR RODOLFO BARILI - Hola. Buenas noches...


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1145


### 1 - Preprocesamiento

In [5]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
sentence_tokens = []

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cbureu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
for _, row in df.iterrows():
    tokens = word_tokenize(row[0])
    sentence_tokens.append(tokens)

In [7]:
sentence_tokens[:2]

[['TEXTO',
  'DEL',
  'PRIMER',
  'DEBATE',
  'PRESIDENCIAL',
  '2023',
  ',',
  'REALIZADO',
  'EN',
  'EL',
  'CENTRO',
  'DE'],
 ['CONVENCIONES',
  'PROVINCIAL',
  'DE',
  'FORUM',
  ',',
  'CON',
  'SEDE',
  'EN',
  'LA',
  'CIUDAD',
  'DE',
  'SANTIAGO',
  'DEL']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Parámetros a probar
vocab_sizes = [1000, 5000]  # Diferentes tamaños de vocabulario
window_sizes = [2, 5] # Diferentes tamaños de ventana de contexto
best_loss = float('inf')  
best_model = None

# Entrenamiento del modelo Word2Vec con distintos tamaños de vocabulario y ventana de contexto
for vocab_size in vocab_sizes:
    for window_size in window_sizes:
        for sg_value in [0, 1]:  # Probamos con ambos modelos: CBOW (0) y Skip-gram (1)
            for negative_value in [10, 20]:  # Probamos diferentes valores de negative samples
                    # Crear y entrenar el modelo Word2Vec con los parámetros especificados
                    w2v_model = Word2Vec(min_count=5,    # Frecuencia mínima de palabra para incluirla en el vocabulario
                                         window=window_size,       # Tamaño de la ventana de contexto
                                         vector_size=300,       # Dimensionalidad de los vectores
                                         negative=negative_value,    # Cantidad de negative samples
                                         workers=1,      # Si tienes más cores puedes cambiar este valor
                                         sg=sg_value)           # Modelo 0: CBOW, 1: Skip-gram
                                        
                    # Construir el vocabulario
                    w2v_model.build_vocab(sentence_tokens[:vocab_size])  # Utilizar un subconjunto del corpus para el tamaño del vocabulario
                    
                    # Entrenar el modelo
                    w2v_model.train(sentence_tokens[:vocab_size],
                                    total_examples=w2v_model.corpus_count,
                                    epochs=50,  # Número de épocas
                                    compute_loss=True)  # Calcular la pérdida
                    
                    if w2v_model.get_latest_training_loss() < best_loss:
                        best_loss = w2v_model.get_latest_training_loss()
                        best_model = w2v_model
                                    
                    # Imprimir los parámetros y la pérdida final
                    print(f"Modelo con vocab_size={vocab_size}, window_size={window_size}, sg={sg_value}, negative={negative_value}:")
                    print("Loss:", w2v_model.get_latest_training_loss())


Modelo con vocab_size=1000, window_size=2, sg=0, negative=10:
Loss: 1078189.25
Modelo con vocab_size=1000, window_size=2, sg=0, negative=20:
Loss: 1267350.0
Modelo con vocab_size=1000, window_size=2, sg=1, negative=10:
Loss: 2417797.5
Modelo con vocab_size=1000, window_size=2, sg=1, negative=20:
Loss: 2726326.75
Modelo con vocab_size=1000, window_size=5, sg=0, negative=10:
Loss: 1087306.25
Modelo con vocab_size=1000, window_size=5, sg=0, negative=20:
Loss: 1280357.125
Modelo con vocab_size=1000, window_size=5, sg=1, negative=10:
Loss: 3857837.0
Modelo con vocab_size=1000, window_size=5, sg=1, negative=20:
Loss: 4343940.0
Modelo con vocab_size=5000, window_size=2, sg=0, negative=10:
Loss: 1241031.25
Modelo con vocab_size=5000, window_size=2, sg=0, negative=20:
Loss: 1448447.375
Modelo con vocab_size=5000, window_size=2, sg=1, negative=10:
Loss: 2795463.0
Modelo con vocab_size=5000, window_size=2, sg=1, negative=20:
Loss: 3150270.0
Modelo con vocab_size=5000, window_size=5, sg=0, negativ

Parece que los modelos con menor pérdida final son aquellos con sg=0 (CBOW) en lugar de sg=1 (Skip-gram) y negative=10 en lugar de negative=20

In [10]:

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", best_model.corpus_count)

Cantidad de docs en el corpus: 1000


In [11]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(best_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 435


### 4 - Ensayar

In [12]:
# Palabras que MÁS se relacionan con...:
best_model.wv.most_similar(positive=["Milei"], topn=10)

[('responde', 0.8193460702896118),
 ('preguntar', 0.7511956095695496),
 ('soy', 0.7002146244049072),
 ('cuatro', 0.6935486197471619),
 ('bien', 0.6835799813270569),
 ('hecho', 0.6759225130081177),
 ('solicitado', 0.6754158139228821),
 ('ha', 0.6696041226387024),
 ('palabra', 0.6640872955322266),
 ('plantea', 0.6624125838279724)]

In [13]:

# Palabras que MÁS se relacionan con...:
best_model.wv.most_similar(positive=["SERGIO"], topn=10)

[('MYRIAM', 0.9545221924781799),
 ('JAVIER', 0.9383963346481323),
 ('BREGMAN', 0.9230508208274841),
 ('JUAN', 0.9157273769378662),
 ('PATRICIA', 0.90707927942276),
 ('Buenas', 0.9054704308509827),
 ('Muy', 0.9042426347732544),
 ('MASSA', 0.8734833002090454),
 ('Sí', 0.8713484406471252),
 ('noches', 0.8701626062393188)]

In [14]:
# Palabras que MÁS se relacionan con...:
best_model.wv.most_similar(positive=["Córdoba"], topn=10)

[('esto', 0.862898051738739),
 ('tenemos', 0.8383234739303589),
 ('también', 0.8348739147186279),
 ('necesita', 0.822354257106781),
 ('realidad', 0.8131127953529358),
 ('peor', 0.8106259703636169),
 ('además', 0.8036535382270813),
 ('trabajo', 0.8006067872047424),
 ('otros', 0.7995463609695435),
 ('argentino', 0.7986412048339844)]

In [15]:
# Palabras que MÁS se relacionan con...:
best_model.wv.most_similar(positive=["Bullrich"], topn=10)

[('preguntar', 0.7550298571586609),
 ('palabra', 0.7540703415870667),
 ('responde', 0.7343611717224121),
 ('pregunta', 0.71916663646698),
 ('exposición', 0.6980674266815186),
 ('Bregman', 0.6777220964431763),
 ('hablando', 0.6727942824363708),
 ('escuchar', 0.6700204610824585),
 ('pobreza', 0.6635985970497131),
 ('cinco', 0.6610088348388672)]

In [16]:
# Palabras que MÁS se relacionan con...:
best_model.wv.most_similar(positive=["inflación"], topn=10)

[('sin', 0.9165001511573792),
 ('con', 0.9052760004997253),
 ('puede', 0.889393150806427),
 ('donde', 0.8716874718666077),
 ('terminar', 0.8704063892364502),
 ('Cuando', 0.8638389706611633),
 ('política', 0.8575689196586609),
 ('acuerdo', 0.8563706278800964),
 ('moneda', 0.8481765985488892),
 ('sino', 0.8458088636398315)]

In [17]:
# Palabras que MÁS se relacionan con...:
best_model.wv.most_similar(positive=["FMI"], topn=10)

[('dólar', 0.9640938639640808),
 ('PBI', 0.9574315547943115),
 ('Congreso', 0.9529827833175659),
 ('Monetario', 0.9528132677078247),
 ('valor', 0.949656069278717),
 ('deuda', 0.9476145505905151),
 ('mundo', 0.9453921914100647),
 ('AMBA', 0.943834125995636),
 ('actual', 0.9434700608253479),
 ('mejor', 0.9411595463752747)]

### 5 - Visualizar agrupación de vectores

In [18]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [19]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

In [20]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show()

In [21]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/

vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Conclusiones

* El algoritmo es capaz de separar la transcripción del debate presidencial en oraciones y tokenizar las palabras. 
* El modelo de embeddings entrenado con el corpus del debate presidencial es capaz de capturar relaciones semánticas relevantes y contextualizar adecuadamente los términos clave discutidos durante el evento.
* Las palabras asociadas con nombres propios como "Bullrich" y "Sergio" muestran una asociación con otros nombres y saludos, lo que indica que el modelo ha aprendido las relaciones entre diferentes personas que participaron en el debate.
* Las palabras que se relacionan más estrechamente con términos clave como "FMI" y "inflación" reflejan aspectos relevantes del contexto del debate, como "deuda", "PBI", "monetario" para "FMI", y "política", "acuerdo", "moneda" para "inflación". Esto indica que el modelo ha sido capaz de capturar el contexto en el que se utilizaron estas palabras durante el debate.

